# Importing Libraries

In [1]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import SelectFromModel

import joblib

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from flask import Flask, render_template, request


# Reading Data

In [2]:
pd.set_option('display.max_columns',None, 'display.max_colwidth', None)

In [3]:
df = pd.read_excel("divorce.xlsx")
ques = pd.read_excel("Divorce Questions.xlsx", header = None)

In [4]:
df.head(5)

,Atr1,Atr2,Atr3,Atr4,Atr5,Atr6,Atr7,Atr8,Atr9,Atr10,Atr11,Atr12,Atr13,Atr14,Atr15,Atr16,Atr17,Atr18,Atr19,Atr20,Atr21,Atr22,Atr23,Atr24,Atr25,Atr26,Atr27,Atr28,Atr29,Atr30,Atr31,Atr32,Atr33,Atr34,Atr35,Atr36,Atr37,Atr38,Atr39,Atr40,Atr41,Atr42,Atr43,Atr44,Atr45,Atr46,Atr47,Atr48,Atr49,Atr50,Atr51,Atr52,Atr53,Atr54,Class
0,2,2,4,1,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,2,1,2,0,1,2,1,3,3,2,1,1,2,3,2,1,3,3,3,2,3,2,1,1
1,4,4,4,4,4,0,0,4,4,4,4,3,4,0,4,4,4,4,3,2,1,1,0,2,2,1,2,0,1,1,0,4,2,3,0,2,3,4,2,4,2,2,3,4,2,2,2,3,4,4,4,4,2,2,1
2,2,2,2,2,1,3,2,1,1,2,3,4,2,3,3,3,3,3,3,2,1,0,1,2,2,2,2,2,3,2,3,3,1,1,1,1,2,1,3,3,3,3,2,3,2,3,2,3,1,1,1,2,2,2,1
3,3,2,3,2,3,3,3,3,3,3,4,3,3,4,3,3,3,3,3,4,1,1,1,1,2,1,1,1,1,3,2,3,2,2,1,1,3,3,4,4,2,2,3,2,3,2,2,3,3,3,3,2,2,2,1
4,2,2,1,1,1,1,0,0,0,0,0,1,0,1,1,1,1,1,2,1,1,0,0,0,0,2,1,2,1,1,1,1,1,1,0,0,0,0,2,1,0,2,3,0,2,2,1,2,3,2,2,2,1,0,1


# Splitting Test and Train Data

In [5]:
x = df.iloc[:,0:53]
y = df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33, random_state=42)

# Feature Selection using KBest Algorithm

In [11]:
fs = SelectKBest(score_func=chi2, k= 10)

fs.fit(X_train, y_train)

X_train_fs = fs.transform(X_train)
X_test_fs = fs.transform(X_test)



scores = pd.DataFrame(fs.scores_)
names = pd.DataFrame(X_train.columns)








features = pd.concat([names,scores],axis=1)
features.columns = ["names", "scores"]


    
features = features.sort_values(by= ['scores'],ascending=False)

features = features.head(10)

questions = []

for _ in range(len(features)):
    mask  = features.names.iloc[_]
    questions.append(ques[0].loc[ques[1] == mask])

    
    questions = pd.Series(questions)    
questions = pd.DataFrame(questions)


features

,names,scores
35,Atr36,183.797065
34,Atr35,181.106124
39,Atr40,174.375947
37,Atr38,160.588721
8,Atr9,160.425659
18,Atr19,155.229166
16,Atr17,152.548781
10,Atr11,151.993322
4,Atr5,150.218797
19,Atr20,144.605114


# Creating DataFrame of Selected Features

In [12]:
df1 = pd.DataFrame()

index = len(features.index)

for column in range(index):
    
    mask = features.iloc[column]
    df1[mask[0]] = df[mask[0]]
    df1["Class"] = df["Class"]
    
df1   

,Atr36,Class,Atr35,Atr40,Atr38,Atr9,Atr19,Atr17,Atr11,Atr5,Atr20
0,1,1,0,3,1,0,0,0,1,0,1
1,2,1,0,4,4,4,3,4,4,4,2
2,1,1,1,3,1,1,3,3,3,1,2
3,1,1,1,4,3,3,3,3,4,3,4
4,0,1,0,1,0,0,2,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
165,0,0,0,0,0,0,0,0,0,0,0
166,1,0,1,1,1,0,0,0,0,0,0
167,0,0,0,2,1,0,0,0,0,0,1
168,0,0,0,1,0,0,0,0,0,0,0


# Splitting the Final Dataframe

In [13]:
x = df1.drop("Class", axis = 1)
y = df1["Class"]

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.30, random_state=15)

## Naive Bayes

In [14]:
mnb = MultinomialNB().fit(X_train, y_train) #MultiNomial Naive Bayes

print("score on test: " + str(mnb.score(X_test, y_test)))
print("score on train: "+ str(mnb.score(X_train, y_train)))

score on test: 0.6470588235294118
score on train: 0.7058823529411765


## Logistic Regression

In [15]:
lr=LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

print("score on test: " + str(lr.score(X_test, y_test)))
print("score on train: "+ str(lr.score(X_train, y_train)))

score on test: 0.9803921568627451
score on train: 0.9915966386554622


## K - Nearest Neighbour

In [16]:
knn = KNeighborsClassifier(algorithm = 'brute', n_jobs=-1)

knn.fit(X_train, y_train)

print("train shape: " + str(X_train.shape))

print("score on test: " + str(knn.score(X_test, y_test)))
print("score on train: "+ str(knn.score(X_train, y_train)))

train shape: (119, 10)
score on test: 0.9607843137254902
score on train: 0.9831932773109243


## Support Vector Machine

In [17]:
svm=LinearSVC(C=0.0001)
svm.fit(X_train, y_train)

print("score on test: " + str(svm.score(X_test, y_test)))
print("score on train: "+ str(svm.score(X_train, y_train)))

score on test: 0.6274509803921569
score on train: 0.7226890756302521


## Decision Tree Classifier

In [18]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

print("score on test: "  + str(clf.score(X_test, y_test)))
print("score on train: " + str(clf.score(X_train, y_train)))

score on test: 0.9607843137254902
score on train: 1.0


## Bagging Classifier

In [19]:
# maX_samples: maXimum size 0.5=50% of each sample taken from the full dataset
# maX_features: maXimum of features 1=100% taken here all 10K 
# n_estimators: number of decision trees


bg = BaggingClassifier(DecisionTreeClassifier(),max_samples=0.5,max_features=1.0,n_estimators=10)

bg.fit(X_train, y_train)

print("score on test: " + str(bg.score(X_test, y_test)))
print("score on train: "+ str(bg.score(X_train, y_train)))

score on test: 0.9607843137254902
score on train: 1.0


## Random Forest Classifier

In [20]:
# n_estimators = number of decision trees

rf = RandomForestClassifier(n_estimators=10, max_depth=2)
rf.fit(X_train, y_train)

print("score on test: " + str(rf.score(X_test, y_test)))
print("score on train: "+ str(rf.score(X_train, y_train)))



score on test: 0.9607843137254902
score on train: 0.9831932773109243


## Creating Confusion Matrix

In [21]:
y_pred = rf.predict(X_test)
score = rf.score(X_test, y_pred)

print(score, y_pred)

confusion_matrix(y_test, y_pred)

1.0 [1 1 0 0 0 0 0 1 1 0 0 1 0 1 1 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1
 0 0 0 1 0 1 0 0 0 1 1 0 1 0]


array([[28,  0],
       [ 2, 21]])

## Saving the Model

In [22]:
joblib_file = "random_forest_divorce_predictor.pkl"
joblib.dump(rf, joblib_file)


['random_forest_divorce_predictor.pkl']

In [23]:
# Load from file
joblib_model = joblib.load(joblib_file)

In [24]:
score = joblib_model.score(X_test, y_test)
print("Test score: {0:.2f} %".format(100 * score))

Test score: 96.08 %


In [25]:
a = rf.predict([[4,4,4,3,1,4,1,4,2,0]])

In [26]:
a[0]

1

In [27]:
model = joblib.load('random_forest_divorce_predictor.pkl')

In [31]:
prediction = model.predict([[4,4,4,4,4,4,4,4,4,4]])

In [32]:
prediction[0]

1